# Project SMAP data onto Landsat data and save as raster stack

## Objectives

- Read SMAP data from h5 file
- Create raster
- Create rasters from Landsat data
- Project SMAP data onto Landsat data
- Create raster stack containing bands from Landsat data and projection of SMAP data

## Prerequisites

Necessary libraries- 
    - rhdf5
    - raster
    - rdgal
    - sp
Install these packages using the following code

In [ ]:
install.packages("sp",repos="http://cran.rstudio.com/")
install.packages("raster",repos="http://cran.rstudio.com/")
source("https://bioconductor.org/biocLite.R")
biocLite("rgdal")
biocLite("rhdf5")

If you already have these packages installed confirm that you are using the most up-to-date versions

In [ ]:
update.packages("sp",repos="http://cran.rstudio.com/")
update.packages("raster",repos="http://cran.rstudio.com/")
source("https://bioconductor.org/biocLite.R")
biocLite("rgdal")
biocLite("rhdf5")

## Getting Data

### SMAP

To access the SMAP data call download.file, with the first argument being the url where the data is located, and the second being the file name and path where you want to store the data this can be repeated with as many files as are needed

In [4]:
download.file('ftp://n5eil01u.ecs.nsidc.org/SAN/SMAP/SPL4SMGP.001/2015.04.01/SMAP_L4_SM_gph_20150401T013000_Vb1010_001.h5',
             'SMAP_1.h5')

### Landsat

To acquire landsat data you will need the landsat-util command line utility. Information on installation can be found here http://landsat-util.readthedocs.io/en/latest/installation.html If you are using windows be sure to first install docker, and follow the instructions under the docker section for landsat-util installation in the docker terminal. 

Once you have installed landsat-util,] you can use the landsat commands described here http://landsat-util.readthedocs.io/en/latest/overview.html, to search for and download landsat data. Remember if you are using windows to proceed each landsat command with 

docker run -it -v ~/landsat:/root/landsat developmentseed/landsat-util:latest

## Reading SMAP

Include necessary packages, and find the dataset(s) you would like to read in

In [1]:
library(sp)
library(rhdf5)
library(raster)
library(rgdal)

#File name and group where data is located
input_file=paste0('SMAP.h5')#file name and path
group='Geophysical_Data'
h5f=H5Fopen(input_file)
h5f&'Geophysical_Data'#Print out all datasets in the geophysical data group

Warning message:
: package 'raster' was built under R version 3.2.5Loading required package: sp
Warning message:
: package 'sp' was built under R version 3.2.5Warning message:
: package 'rgdal' was built under R version 3.2.5rgdal: version: 1.1-10, (SVN revision 622)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.0.1, released 2015/09/15
 Path to GDAL shared files: C:/Users/Zach/Anaconda3/R/library/rgdal/gdal
 GDAL does not use iconv for recoding strings.
 Loaded PROJ.4 runtime: Rel. 4.9.1, 04 March 2015, [PJ_VERSION: 491]
 Path to PROJ.4 shared files: C:/Users/Zach/Anaconda3/R/library/rgdal/proj
 Linking to sp version: 1.2-3 


Define datasets and read data from h5 file and create raster

In [ ]:
#Define datasets
dataset='sm_rootzone'#possible set to read in
missing_vals=-9999
#read h5 file and get data from desired set
input <- h5read(input_file, paste0('/', group, '/', dataset))
#exclude missing values
input[input == missing_vals] <- NA
#create raster(transpose to )
r <- raster(t(input))

Create vectors of longitude and latitude and define extent for projection using these ranges

In [ ]:
#create vector of lat/lon values
lat <- apply(h5read(input_file, '/cell_lat'), 2, unique)
lon <- apply(h5read(input_file, '/cell_lon'), 2, unique)
#geographical crs input
crs = '+proj=longlat +lat_0=0 +lon_0=0 +lat_ts=30 
                +ellps=WGS84 +datum=WGS84 +units=m'
crs_in = '+proj=cea +lat_0=0 +lon_0=0 +lat_ts=30 
                +ellps=WGS84 +datum=WGS84 +units=m'
#create raster using lon/lat and create projection using crs_in to 
#find extent of data
ex <- extent(projectExtent(raster(extent(range(lon), range(lat)),
                                    crs),
                                    crs_in))

Set extent and projection of raster using previously defined extent and crs_in for projection

In [ ]:
extent(r) <- ex
projection(r) <- CRS(crs_in)

Create final SMAP raster

In [ ]:
crs_out = "+init=epsg:4326"#defines global coordinate reference system
smap <- projectRaster(r, crs = crs_out)
#output raster to geotiff file
writeRaster(smap, 'smap_raw.tif',
            format = 'GTiff', overwrite = TRUE)

## Process and create raster stack

Change allBands if you want to use all available bands

In [ ]:
allBands=FALSE#true if using all bands

If using specific bands create a list with just the bands being used

In [ ]:
#if using specific bands
if(!allBands){
    bands<-c(2,3,4,5,9)#possible selection of bands
}

If using all bands list will contain all available bands

In [ ]:
#if using all bands
if(allBands){
    numbands=9#?
    bands<-range(1,numbands)
}

Loop through list of bands (works with all or specific bands) and create raster from each geotiff file then add raster to list to be added to stack

In [ ]:
#standard lsat file name contains the seceneID_band number
sceneID='LC80330322015099LGN00'
lsat_file=paste0(sceneID,'_B')
#list of each raster to be contained in stack (all bands plus SMAP)
x=vector("list",length(bands)+1)
j=1
for(i in bands){
  if(file.exists(paste0('R/data/',lsat_file,i,'.tif'))){
    x[[j]] <-raster(paste0('R/data/',lsat_file,i,'.tif'))#read file
  }else{
    x[[j]] <-NULL  #if file doesn't exist exclude band from stack
  }
  j=j+1
}

Project SMAP onto lsat. By including the lsat raster that smap will be projected to, there is no need to include crs or res values. Next mask smap to lsat. This takes missing values in smap (set to NA earlier) and masks them with the values in lsat data

In [ ]:
#match resolution of smap and lsat data
smap <-projectRaster(smap, to = x[[1]])
#match extent
smap <-mask(smap,x[[1]])
x[[j]] <-smap

Create and save raster stack to a geotiff file

In [ ]:
#create raster stack
stck <-stack(x)
#save raster stack to a geotiff file
writeRaster(stck,'lsat_smap_stack.tif', format = 'GTiff', overwrite=TRUE)

The raster library has many useful functions that allow you to better understand the data stored in the raster stack. The names function for example allows you to see the what is in each layer, ad change layer names to make this more clear. addLayer and dropLayer allow you to add a layer to or remove a layer from the stack